In [23]:
#IMPORT THE LIBRARIES
import librosa
import numpy as np
from tensorflow.keras.models import model_from_json


In [24]:
from tensorflow.keras.models import Sequential, model_from_json
json_file = open('CNN_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("trial/best_model1_weights.h5")
print("Loaded model from disk")

Loaded model from disk


In [25]:
import pickle

with open('trial/scaler2.pickle', 'rb') as f:
    scaler2 = pickle.load(f)
    
with open('trial/encoder2.pickle', 'rb') as f:
    encoder2 = pickle.load(f)   
print("Done")  

Done


In [26]:
def zcr(data,frame_length,hop_length):
    zcr=librosa.feature.zero_crossing_rate(data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(zcr)
def rmse(data,frame_length=2048,hop_length=512):
    rmse=librosa.feature.rms(y=data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(rmse)
def mfcc(data,sr,frame_length=2048,hop_length=512,flatten:bool=True):
    mfcc=librosa.feature.mfcc(y=data,sr=sr)
    return np.squeeze(mfcc.T)if not flatten else np.ravel(mfcc.T)

def extract_features(data, sr=22050, frame_length=2048, hop_length=512):
    result = np.array([])
    
    zcr_result = zcr(data, frame_length, hop_length)
    print(f"Size after zero crossing rate: {zcr_result.size}")
    
    rmse_result = rmse(data, frame_length, hop_length)
    print(f"Size after RMSE: {rmse_result.size}")
    
    mfcc_result = mfcc(data, sr, frame_length, hop_length)
    print(f"Size after MFCC: {mfcc_result.size}")
    
    result = np.hstack((result, zcr_result, rmse_result, mfcc_result))
    
    return result


In [27]:
def get_predict_feat(path):
    d, s_rate = librosa.load(path, duration=2.5, offset=0.6)
    res = extract_features(d)
    result = np.array(res)

    # Use -1 to automatically calculate the size of the first dimension
    result = np.reshape(result, newshape=(1, -1))

    i_result = scaler2.transform(result)
    final_result = np.expand_dims(i_result, axis=2)

    return final_result


In [28]:
emotions1={1:'Neutral', 2:'Calm', 3:'Happy', 4:'Sad', 5:'Angry', 6:'Fear', 7:'Disgust',8:'Surprise'}
def prediction(path1):
    res=get_predict_feat(path1)
    predictions=loaded_model.predict(res)
    y_pred = encoder2.inverse_transform(predictions)
    return y_pred[0][0]

In [29]:
res=get_predict_feat("C:/Users/ADMIN/Downloads/03-01-07-01-01-01-01.wav")
print(res.shape)

Size after zero crossing rate: 108
Size after RMSE: 108
Size after MFCC: 2160
(1, 2376, 1)


In [30]:
prediction(r"Dataset\Ravdess\audio_speech_actors_01-24\Actor_01\03-01-02-02-01-01-01.wav")

Size after zero crossing rate: 108
Size after RMSE: 108
Size after MFCC: 2160
1/1 [==============================] - 0s 221ms/step


'neutral'

In [31]:
prediction(r"Dataset\Ravdess\audio_speech_actors_01-24\Actor_01\03-01-03-01-02-02-01.wav")

Size after zero crossing rate: 108
Size after RMSE: 108
Size after MFCC: 2160
1/1 [==============================] - 0s 53ms/step


'happy'

In [32]:
prediction(r"C:\Users\ADMIN\Downloads\WhatsApp-Audio-2023-10-30-at-15.29.36.waptt.wav")

Size after zero crossing rate: 108
Size after RMSE: 108
Size after MFCC: 2160
1/1 [==============================] - 0s 56ms/step


'angry'

In [33]:
import gradio as gr
import librosa
import numpy as np
from tensorflow.keras.models import model_from_json
import pickle

# Load your model and other necessary files here...


# Your functions and model loading code here...


iface = gr.Interface(
    fn=prediction,
    inputs=gr.Audio(type='filepath'),
    outputs="text",
    live=True,
    title="Speech Emotion Recognition",
    description="Upload an audio file and get the predicted emotion label."
)

# Launch Gradio Interface
iface.launch()




Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Size after zero crossing rate: 37
Size after RMSE: 37
Size after MFCC: 740


Traceback (most recent call last):
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\queueing.py", line 456, in call_prediction
    output = await route_utils.call_process_api(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1522, in process_api
    result = await self.call_function(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1144, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packa

Size after zero crossing rate: 108
Size after RMSE: 108
Size after MFCC: 2160
1/1 [==============================] - 0s 74ms/step


Traceback (most recent call last):
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\queueing.py", line 456, in call_prediction
    output = await route_utils.call_process_api(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1522, in process_api
    result = await self.call_function(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1144, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packa

Size after zero crossing rate: 108
Size after RMSE: 108
Size after MFCC: 2160
1/1 [==============================] - 0s 86ms/step


Traceback (most recent call last):
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\queueing.py", line 456, in call_prediction
    output = await route_utils.call_process_api(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1522, in process_api
    result = await self.call_function(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1144, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packa

Size after zero crossing rate: 108
Size after RMSE: 108
Size after MFCC: 2160
1/1 [==============================] - 0s 190ms/step
